# Importando as bibliotecas

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from collections import Counter
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

In [2]:
# Dataset

In [3]:
procData = ProcessarDados("../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv")

In [4]:
procData

In [5]:
# Funções

In [6]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_f1 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        X_train, X_test, y_train, y_test = procData.holdout(0.2)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor K-Vizinhos, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        clf = KNeighborsClassifier()
        grid_knn = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy')
        grid_knn.fit(X_train, y_train)
        
        # Treinando do modelo com os melhores parametros encontrados
        grid_knn.best_params_
        Kvizinhos_best = int(grid_knn.best_params_['n_neighbors']) 
        
        KNN = KNeighborsClassifier(n_neighbors=Kvizinhos_best, metric='minkowski', p = 2)
        KNN.fit(X_train, y_train)

        #testando o modelo
        y_pred = KNN.predict(X_test)
        if exibir_matriz_confusao:
            print(confusion_matrix(y_test, y_pred))

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        f1_score = 2 * (precision * recall) / (precision + recall)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_f1.append(f1_score)

        best_parametros = "K-Vizinhos: "+ str(Kvizinhos_best);
        resultados_parametros.append(best_parametros)


        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("F1-Score:",f1_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['f1'] = resultados_f1
    
    return resultados_gerais

In [7]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False):
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\t F1-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f1'], axis=0), np.std(dict_metricas['f1'], axis=0)))
        print ("====================================================================================================")

# Definindo os parâmetros

In [8]:
c_range = np.arange(1,25)
param_grid = {'n_neighbors': c_range}
epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True


# Treinando e obtendo as métricas do modelo

In [9]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('KNN',dict_metricas)

[[27  8]
 [ 3 82]]
Rodada: # 0
K-Vizinhos: 9
Accuracy: 0.9083333333333333
Precision: 0.9111111111111111
Recall: 0.9647058823529412
F1-Score: 0.9371428571428572


[[20  6]
 [ 8 86]]
Rodada: # 1
K-Vizinhos: 1
Accuracy: 0.8833333333333333
Precision: 0.9347826086956522
Recall: 0.9148936170212766
F1-Score: 0.924731182795699




C:\Users\Eduardo Valente\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[[24  9]
 [ 7 80]]
Rodada: # 2
K-Vizinhos: 5
Accuracy: 0.8666666666666667
Precision: 0.898876404494382
Recall: 0.9195402298850575
F1-Score: 0.9090909090909091


[[24  6]
 [ 1 89]]
Rodada: # 3
K-Vizinhos: 3
Accuracy: 0.9416666666666667
Precision: 0.9368421052631579
Recall: 0.9888888888888889
F1-Score: 0.9621621621621621


[[19  8]
 [ 7 86]]
Rodada: # 4
K-Vizinhos: 1
Accuracy: 0.875
Precision: 0.9148936170212766
Recall: 0.9247311827956989
F1-Score: 0.9197860962566845




C:\Users\Eduardo Valente\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[[17  9]
 [ 3 91]]
Rodada: # 5
K-Vizinhos: 3
Accuracy: 0.9
Precision: 0.91
Recall: 0.9680851063829787
F1-Score: 0.9381443298969071




...

[[25  4]
 [ 5 86]]
Rodada: # 42
K-Vizinhos: 1
Accuracy: 0.925
Precision: 0.9555555555555556
Recall: 0.945054945054945
F1-Score: 0.9502762430939227


[[29  6]
 [ 1 84]]
Rodada: # 43
K-Vizinhos: 1
Accuracy: 0.9416666666666667
Precision: 0.9333333333333333
Recall: 0.9882352941176471
F1-Score: 0.96




C:\Users\Eduardo Valente\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[[19 10]
 [ 8 83]]
Rodada: # 44
K-Vizinhos: 3
Accuracy: 0.85
Precision: 0.8924731182795699
Recall: 0.9120879120879121
F1-Score: 0.9021739130434783




C:\Users\Eduardo Valente\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[[23  9]
 [ 1 87]]
Rodada: # 45
K-Vizinhos: 1
Accuracy: 0.9166666666666666
Precision: 0.90625
Recall: 0.9886363636363636
F1-Score: 0.9456521739130435


[[17 10]
 [ 4 89]]
Rodada: # 46
K-Vizinhos: 11
Accuracy: 0.8833333333333333
Precision: 0.898989898989899
Recall: 0.956989247311828
F1-Score: 0.9270833333333334


[[23 13]
 [ 2 82]]
Rodada: # 47
K-Vizinhos: 5
Accuracy: 0.875
Precision: 0.8631578947368421
Recall: 0.9761904761904762
F1-Score: 0.9162011173184357


[[22  5]
 [ 5 88]]
Rodada: # 48
K-Vizinhos: 11
Accuracy: 0.9166666666666666
Precision: 0.946236559139785
Recall: 0.946236559139785
F1-Score: 0.946236559139785


[[24  3]
 [10 83]]
Rodada: # 49
K-Vizinhos: 4
Accuracy: 0.8916666666666667
Precision: 0.9651162790697675
Recall: 0.8924731182795699
F1-Score: 0.9273743016759777


============================================== KNN =================================================
=================================== TABELA DE MÉTRICAS DO MODELO ===================================
	 Accuracy